In [11]:
import numpy as np
import matplotlib.pyplot as plt
from ESINet.forward import create_forward_model
from ESINet.simulations import run_simulations, create_eeg
from ESINet.util import *
from ESINet.ann import *

pth_fwd = 'forward_models/ico3/'

In [12]:
#create_forward_model(pth_fwd, sampling='ico3')

/Users/moritzmaschke/opt/anaconda3/envs/py_eeg/lib/python3.8/site-packages/ESINet/forward/forward.py:135: RuntimeWarning: An average reference projection was already added. The data has been left untouched.
  evoked.set_eeg_reference('average', projection=True, verbose=0)


    Write a source space...
    [done]
    Write a source space...
    [done]
    2 source spaces written


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:   16.4s remaining:   27.4s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:   16.5s remaining:    9.9s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:   16.7s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:    0.1s remaining:    0.2s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    0.1s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:    0.3s remaining:    0.5s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:    0.3s remaining:    0.2s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    0.3s finished
/Users/moritzmaschke/opt/anaconda3/envs/py_eeg/lib/python3.8/site-packages/ESINet/for

shape of leadfield: (61, 1284)


FileNotFoundError: [Errno 2] No such file or directory: 'forward_models/ico3//ico3-src.fif'

In [13]:
## changed line 95 in forward.py
#source = mne.read_source_spaces(savepath+"\"+sampling+"-src.fif", verbose=verbose) #changed / into \ . mm_2021-05-25


In [15]:
create_forward_model(pth_fwd, sampling='ico3')

/Users/moritzmaschke/opt/anaconda3/envs/py_eeg/lib/python3.8/site-packages/ESINet/forward/forward.py:135: RuntimeWarning: An average reference projection was already added. The data has been left untouched.
  evoked.set_eeg_reference('average', projection=True, verbose=0)


    Write a source space...
    [done]
    Write a source space...
    [done]
    2 source spaces written


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:   17.2s remaining:   28.6s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:   17.2s remaining:   10.3s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:   17.4s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:    0.1s remaining:    0.2s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    0.1s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:    0.3s remaining:    0.5s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:    0.3s remaining:    0.2s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    0.3s finished
/Users/moritzmaschke/opt/anaconda3/envs/py_eeg/lib/python3.8/site-packages/ESINet/for

shape of leadfield: (61, 1284)


FileNotFoundError: [Errno 2] No such file or directory: 'forward_models/ico3//ico3-src.fif'

## Simulate some source activity and EEG data

In [10]:
sources_sim = run_simulations(pth_fwd, durOfTrial=0)
eeg_sim = create_eeg(sources_sim, pth_fwd)

FileNotFoundError: [Errno 2] No such file or directory: 'forward_models/ico3//pos.pkl'

## Plot a simulated sample and corresponding EEG

In [ ]:
%matplotlib qt
sample = 0  # index of the simulation
title = f'Simulation {sample}'
# Topographic plot
eeg_sim[sample].average().plot_topomap([0.5])
# Source plot
sources_sim.plot(hemi='both', initial_time=sample, surface='white', colormap='inferno', title=title, time_viewer=False)

## Load and train ConvDip with simulated data

In [ ]:
# Find out input and output dimensions based on the shape of the leadfield 
input_dim, output_dim = load_leadfield(pth_fwd).shape
# Initialize the artificial neural network model
model = get_model(input_dim, output_dim)
# Train the model
model, history = train_model(model, sources_sim, eeg_sim, delta=1)

## Evaluate ConvDip
Let's evaluate our model!

In [ ]:
%matplotlib qt
# Load some files from the forward model
leadfield = load_leadfield(pth_fwd)
info = load_info(pth_fwd)

# Simulate a brand new sample:
sources_eval = run_simulations(pth_fwd, 1, durOfTrial=0)
eeg_eval = create_eeg(sources_eval, pth_fwd)

# Calculate the ERP (average across trials):
eeg_sample = np.squeeze( eeg_eval )

# Predict
source_predicted = predict(model, eeg_sample, pth_fwd)

# Visualize ground truth...
title = f'Ground Truth'
sources_eval.plot(hemi='both', initial_time=0.5, surface='white', colormap='inferno', title=title, time_viewer=False)

# ... and prediction
title = f'ConvDip Prediction'
source_predicted.plot(hemi='both', initial_time=0.5, surface='white', colormap='inferno', title=title, time_viewer=False)

# ... and the 'True' EEG topography
title = f'Simulated EEG'
eeg_eval[0].average().plot_topomap([0], title=title)